In [1]:
import pyspark.sql.functions as F
import struct
from tqdm import tqdm

In [2]:
raw_data = '/Users/IsabelGontijo/git/covid-19/raw/data/cases-brazil-cities-time.csv'
tmp = '/Users/IsabelGontijo/temp' 
conv_path = '/Users/IsabelGontijo/git/covid-19/refined/data/01_conversion/' 

In [3]:
!iconv -f utf-8 -t ascii//TRANSLIT $raw_data > $tmp/tmp.csv

In [4]:
!sed "s/[\'\^\~]//g" $tmp/tmp.csv > $tmp/tmp2.csv

In [6]:
df = spark.read.csv(f'{tmp}/tmp2.csv', header=True, multiLine=True)

In [7]:
# List of special characters
special_chars = [struct.pack('B', x).decode() for x in range(1, 32) if x != 10] + ["\n"]
for col in tqdm(df.columns):
    # Replace \t
    df = df.withColumn(col, F.regexp_replace(F.col(col), '\t+', ' ')) # adicionar F.trim
    # Remove special characters with pyspark and upper fields
    df = df.withColumn(col, F.upper(F.regexp_replace(F.col(col), "|".join(special_chars), '')))

100%|██████████| 12/12 [00:02<00:00,  4.23it/s]


In [8]:
df.write.csv(conv_path, header=True, emptyValue='', mode='overwrite')

In [9]:
!rm {tmp}/tmp.csv

In [10]:
if sc: sc.stop()